# A3 Evolving the Graph

In this script, we present the cypher querires corresponding to Option B as described in the PDF report.

## Additional constraints to our new model

### Constrain to enforce that every review relationship has a decision

```bash
CREATE CONSTRAINT reviewedDecisionExists
FOR ()-[r:REVIEWED]-()
REQUIRE r.decision IS NOT NULL;

### Constraint to keep only papers with mostly positive reviews

```bash
// find all papers where more than half of its reviews say “accept”
MATCH (p:article)<-[r:REVIEWED]-()
WITH
  p,
  count(*)                                         AS totalReviews,
  sum(CASE WHEN r.decision = 'accept' THEN 1 ELSE 0 END) AS numAccepts
WHERE numAccepts > totalReviews / 2
RETURN 
  p.key         AS paper,
  numAccepts,
  totalReviews;



## Query to create new REVIEW edges in the evolved graph

```bash
UNWIND [
  { paperKey: 'journals/ral/AhnSD18', reviewerName: 'Feras Dayoub',  decision: 'accept', content: 'Nice write-up!' },
  { paperKey: 'journals/ral/AhnSD18', reviewerName: 'Ho Seok Ahn',   decision: 'reject', content: 'Methods unclear.' },
  { paperKey: 'journals/ral/AhnSD18', reviewerName: 'Inkyu Sa',      decision: 'accept', content: 'Solid evaluation.' }
] AS row

MATCH (p:article {key: row.paperKey})
MATCH (a:author {author: row.reviewerName})

CREATE (p)-[:REVIEWED {
    decision: row.decision,
    content:  row.content,
    reviewedOn: date()
}]->(a);

## Query to Create Affiliations/Organizations

Below, we will futhter evolve to:

- Add a node type Organization (for university, company, etc.).

- Connect authors to organizations using AFFILIATED_WITH.

```bash
UNWIND [
  { authorName: "Feras Dayoub",        org: "Queensland University of Technology" },
  { authorName: "Ho Seok Ahn",         org: "Korea Advanced Institute of Science and Technology" },
  { authorName: "Inkyu Sa",            org: "Kyungpook National University" },
  { authorName: "Jean C. S. Rosa",     org: "Federal University of Technology - Paraná" },
  { authorName: "Thiago Baesso Procaci", org: "Federal University of São Carlos" },
  { authorName: "Eliana Alves Moreira", org: "University of São Paulo" }
] AS row

MATCH (a:author {author: row.authorName})
MERGE (o:Organization {name: row.org})
MERGE (a)-[:AFFILIATED_WITH]->(o);

In [1]:
from IPython.display import HTML
def display_image_with_white_background(img_path): #this function avoids bad looking images and caching problems when using html in jupyter notebook
    html = f"""
    <div style="background-color: white; padding: 10px; display: inline-block; border: 1px solid #ddd; border-radius: 8px;">
      <img src="{img_path}" alt="Graph" />
    </div>
    """
    return HTML(html)

display_image_with_white_background("graph4.png")